# Ego routing

This notebook generates the `min(duration)` route from each conflict to each potential border crossing

In [1]:
import json
import time
import requests

# google libraries
import googlemaps
import polyline

# mapping and shape utils
import folium
from folium import plugins

# data processing
import pandas as pd
import geopandas as gpd

import math

/Users/brandonrose/opt/anaconda3/envs/b39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Google Directions API

First, you need to enable the Google Directions API. You can do this through the Google Developer Console. Once you have a valid API key you should insert it below, in place of `KEY_GOES_HERE`. Note that you should avoid publishing your API key on Github or other public locations.

In [537]:
gmaps = googlemaps.Client(key='KEY_GOES_HERE')

## Processing Conflict Locations

Now we process in conflicts from ACLED. We collapse the conflict locations by filtering for those with more than 10 fatalities. Then we round the latitude and longitude to 1 decimal point (~11km resolution) and groupby this rounded lat/lon.

We select the location name from this group based on the location with the most fatalities.

In [86]:
df_ = pd.read_csv('inputs/ACLED_Conflicts_05.27.22.csv')
df_ = df_[df_['FATALITIES']>10]
df_['lat'] = round(df_.LATITUDE,1)
df_['lon'] = round(df_.LONGITUDE,1)
conflicts = pd.DataFrame(df_.groupby(['lat','lon']).FATALITIES.sum()).reset_index()

In [507]:
df_

,EVENT_DATE,EVENT_TYPE,COUNTRY,LOCATION,LATITUDE,LONGITUDE,FATALITIES,lat,lon
1102,24-Feb-22,Battles,Ukraine,Schastia,48.741,39.235,50,48.7,39.2
1283,26-Feb-22,Explosions/Remote violence,Ukraine,Volnovakha,47.601,37.497,15,47.6,37.5
1290,26-Feb-22,Battles,Ukraine,Oleshky,46.618,32.719,13,46.6,32.7
1297,26-Feb-22,Battles,Ukraine,Kobleve,46.665,31.208,20,46.7,31.2
1298,26-Feb-22,Battles,Ukraine,Kobleve,46.665,31.208,20,46.7,31.2
...,...,...,...,...,...,...,...,...,...
6937,16-May-22,Explosions/Remote violence,Ukraine,Chornobaivka,46.697,32.553,29,46.7,32.6
7018,17-May-22,Explosions/Remote violence,Ukraine,Desna,50.925,30.772,87,50.9,30.8
7214,19-May-22,Explosions/Remote violence,Ukraine,Severodonetsk,48.947,38.491,12,48.9,38.5
7258,19-May-22,Explosions/Remote violence,Ukraine,Luhansk,48.567,39.317,17,48.6,39.3


In [87]:
def get_loc_name(row):
    loc = df_[(df_.lat==row.lat)&(df_.lon==row.lon)]\
                .sort_values(by=['FATALITIES'], ascending=False)\
                .iloc[0].LOCATION
    row['location'] = loc
    return row

conflicts = conflicts.apply(lambda row: get_loc_name(row), axis=1)
conflicts.rename(columns={'FATALITIES': 'fatalities'},inplace=True)
conflicts.head()

,lat,lon,fatalities,location
0,43.4,36.5,37.0,Eastern Black Sea
1,45.3,30.2,42.0,Zmiyinyy Island
2,46.6,32.1,100.0,Oleksandrivka
3,46.6,32.7,13.0,Oleshky
4,46.7,31.2,40.0,Kobleve


Now we read in border crossing data.

We use the crossings data updated on `14 April 2022`

In [111]:
crossings = pd.read_excel('inputs/ukr_border_crossings_140422.xlsx', sheet_name='Border Crossings')

crossings.rename(columns={'Name - English': '#name', 
                          'Country': 'country',
                          'Lat': 'lat',
                          'Long': 'lon'}, inplace=True)

crossings['country'] = crossings.country.apply(lambda x: x.strip())

## Generate Ego Routes

In [248]:
xings_l = list(zip(crossings.lat, crossings.lon))

In [249]:
# Exclude first 2 conflicts since both are in the ocean!
conflicts_l = list(zip(conflicts.lat, conflicts.lon))[2:]

In [250]:
chunk_size = 10
xings_chunked = [xings_l[i:i+chunk_size] for i in range(0,len(xings_l),chunk_size)]
conflicts_chunked = [conflicts_l[i:i+chunk_size] for i in range(0,len(conflicts_l),chunk_size)]

In [346]:
def get_conflict(loc):
    conflict = conflicts[(conflicts.lat==loc[0])&(conflicts.lon==loc[1])].iloc[0]
    return conflict

def get_crossing(loc):
    crossing = crossings[(crossings.lat==loc[0])&(crossings.lon==loc[1])].iloc[0]
    return crossing

def get_conflict_loc(name):
    conflict = conflicts[conflicts['location']==name].iloc[0]
    return conflict

def get_crossing_loc(name):
    crossing = crossings[crossings['#name']==name].iloc[0]
    return crossing

In [341]:
def gen_distance_matrix(mode):
    matrix_dfs = []
    for conflicts_chunk in conflicts_chunked:
        for xings_chunk in xings_chunked:
            res = gmaps.distance_matrix(origins=conflicts_chunk,
                destinations=xings_chunk, mode=mode)

            results = []
            for orig_ix, conf in enumerate(res['rows']):
                conflict = get_conflict(conflicts_chunk[orig_ix])
                for dest_ix, dist in enumerate(conf['elements']):
                    xing = get_crossing(xings_chunk[dest_ix])
                    duration = dist.get('duration', {'value': None, 'text': None})
                    output = dict(conflict=conflict.location,
                                  crossing=xing['#name'],
                                  crossing_country=xing.country,
                                  duration=duration['value'],
                                  duration_text=duration['text'])
                    results.append(output)
            matrix_dfs.append(pd.DataFrame(results))
    return pd.concat(matrix_dfs)

In [351]:
distance_matrix_driving = gen_distance_matrix('driving')

In [342]:
distance_matrix_transit = gen_distance_matrix('transit')

In [406]:
distance_matrix_driving['crossing_lat'] = distance_matrix_driving.crossing.apply(lambda x: get_crossing_loc(x).lat)
distance_matrix_driving['crossing_lon'] = distance_matrix_driving.crossing.apply(lambda x: get_crossing_loc(x).lon)
distance_matrix_driving['mode'] = 'driving'

distance_matrix_transit['crossing_lat'] = distance_matrix_transit.crossing.apply(lambda x: get_crossing_loc(x).lat)
distance_matrix_transit['crossing_lon'] = distance_matrix_transit.crossing.apply(lambda x: get_crossing_loc(x).lon)
distance_matrix_transit['mode'] = 'transit'

distance_matrix_driving['conflict_lat'] = distance_matrix_driving.conflict.apply(lambda x: get_conflict_loc(x).lat)
distance_matrix_driving['conflict_lon'] = distance_matrix_driving.conflict.apply(lambda x: get_conflict_loc(x).lon)

distance_matrix_transit['conflict_lat'] = distance_matrix_transit.conflict.apply(lambda x: get_conflict_loc(x).lat)
distance_matrix_transit['conflict_lon'] = distance_matrix_transit.conflict.apply(lambda x: get_conflict_loc(x).lon)

In [407]:
# combined distance matrixes by first swapping all NONE results from the transit for the corresponding
# conflict, crossing pair from the driving distance matrix
transit_none = distance_matrix_transit[distance_matrix_transit.duration.isna()]
transit_none = transit_none[['conflict','crossing']]
transit_none = pd.merge(transit_none, distance_matrix_driving, how='left', left_on=['conflict','crossing'], right_on=['conflict','crossing'])
transit = distance_matrix_transit[~distance_matrix_transit.duration.isna()]
distance_matrix_combined = pd.concat([transit,transit_none])
distance_matrix_combined = distance_matrix_combined.sort_values(by=['conflict','crossing'])

In [488]:
distance_matrix_driving.to_csv('outputs/distance_matrix_driving.csv', index=False)
distance_matrix_transit.to_csv('outputs/distance_matrix_transit.csv', index=False)

In [412]:
distance_matrix_combined.to_csv('outputs/distance_matrix_combined.csv', index=False)

In [527]:
# distance_matrix_all includes all transit and driving routes
# we should use this for route selection since we really want to include 2 routes if transit exists
# these are: the best driving and the best transit routes
# then we can let the linear program decide which to use based on duration

distance_matrix_all = pd.concat([distance_matrix_transit.dropna(), distance_matrix_driving])
distance_matrix_all.to_csv('outputs/distance_matrix_all.csv', index=False)

In [528]:
distance_matrix_combined = distance_matrix_combined.sort_values(by=['conflict','crossing_country','duration'])
distance_matrix_all = distance_matrix_all.sort_values(by=['conflict','crossing_country','duration','mode'])

In [529]:
selected_routes = []
for conflict in conflict_locations[2:]:
    for country in crossings.country.unique():

        matches = distance_matrix_all[(distance_matrix_all.conflict==conflict)&\
                         (distance_matrix_all.crossing_country==country)]

        # obtain transit route if it exists
        if 'transit' in matches['mode'].unique():
            m = matches[matches['mode']=='transit']
            m = m.sort_values('duration').iloc[0]
            selected_routes.append(m)

        # always get the driving route
        m = matches[matches['mode']=='driving']
        m = m.sort_values('duration').iloc[0]
        selected_routes.append(m)            
        
routes = pd.DataFrame(selected_routes)

In [530]:
routes.shape

(406, 10)

In [531]:
routes['fatalities'] = routes.conflict.apply(lambda x: get_conflict_loc(x).fatalities)

In [532]:
routes.to_csv('outputs/selected_routes.csv', index=False)

### Get directions for routes:

In [553]:
# Get directions for transit routes
directions = []
for kk, route in routes.iterrows():    
    conflict_name = route.conflict
    print(f"{conflict_name} -> {route.crossing_country}")
    try:
        directions_result = gmaps.directions([route.conflict_lat, route.conflict_lon],
                                             [route.crossing_lat, route.crossing_lon],
                                         mode=route['mode'])
        directions_result[0]['conflict']=route.conflict
        directions_result[0]['crossing']=route.crossing
        directions_result[0]['crossing_country']=route.crossing_country
        directions_result[0]['mode']=route['mode']
    except Exception as e:
        print(e)
        directions_result = None
    
    d_ = route.to_dict()
    d_['mode'] = route['mode']
    d_['directions'] = directions_result
    with open('outputs/directions_dict.jsonl', 'a') as f:
        f.write(json.dumps(d_)+'\n')
    
    directions.append(d_)

Oleksandrivka -> Poland
Oleksandrivka -> Moldova
Oleksandrivka -> Romania
Oleksandrivka -> Slovakia
Oleksandrivka -> Hungary
Oleksandrivka -> Belarus
Oleksandrivka -> Russian Federation
Oleshky -> Poland
Oleshky -> Moldova
Oleshky -> Romania
Oleshky -> Slovakia
Oleshky -> Hungary
Oleshky -> Belarus
Oleshky -> Russian Federation
Kobleve -> Poland
Kobleve -> Moldova
Kobleve -> Romania
Kobleve -> Slovakia
Kobleve -> Hungary
Kobleve -> Belarus
Kobleve -> Russian Federation
Kherson -> Poland
Kherson -> Poland
Kherson -> Moldova
Kherson -> Moldova
Kherson -> Romania
Kherson -> Romania
Kherson -> Slovakia
Kherson -> Slovakia
Kherson -> Hungary
Kherson -> Hungary
Kherson -> Belarus
Kherson -> Russian Federation
Kherson -> Russian Federation
Kyselivka -> Poland
Kyselivka -> Moldova
Kyselivka -> Romania
Kyselivka -> Slovakia
Kyselivka -> Hungary
Kyselivka -> Belarus
Kyselivka -> Russian Federation
Melitopol -> Poland
Melitopol -> Moldova
Melitopol -> Romania
Melitopol -> Slovakia
Melitopol -> Hu

Hostomel -> Poland
Hostomel -> Poland
Hostomel -> Moldova
Hostomel -> Moldova
Hostomel -> Romania
Hostomel -> Romania
Hostomel -> Slovakia
Hostomel -> Slovakia
Hostomel -> Hungary
Hostomel -> Hungary
Hostomel -> Belarus
Hostomel -> Belarus
Hostomel -> Russian Federation
Hostomel -> Russian Federation
Desna -> Poland
Desna -> Moldova
Desna -> Romania
Desna -> Slovakia
Desna -> Hungary
Desna -> Belarus
Desna -> Russian Federation
Sumy -> Poland
Sumy -> Poland
Sumy -> Moldova
Sumy -> Moldova
Sumy -> Romania
Sumy -> Romania
Sumy -> Slovakia
Sumy -> Slovakia
Sumy -> Hungary
Sumy -> Belarus
Sumy -> Belarus
Sumy -> Russian Federation
Sumy -> Russian Federation
Kruty -> Poland
Kruty -> Moldova
Kruty -> Romania
Kruty -> Slovakia
Kruty -> Hungary
Kruty -> Belarus
Kruty -> Russian Federation
Chernihiv -> Poland
list index out of range
Chernihiv -> Poland
Chernihiv -> Moldova
Chernihiv -> Moldova
Chernihiv -> Romania
Chernihiv -> Romania
Chernihiv -> Slovakia
Chernihiv -> Slovakia
Chernihiv -> Hun

In [580]:
# now we have to fix some missing records that were transit routes
# available on 5/29 not showing up on 5/31/22

directions_old = []
with open('outputs/directions.jsonl','r') as f:
    for line in f:
        directions_old.append(json.loads(line.split('\n')[0]))
        
missing = []
for d in directions:
    if not d['directions']:
        for i in directions_old:
            if i[0]['conflict'] == d['conflict'] and i[0]['crossing'] == d['crossing']:
                d['directions'] = i

In [590]:
with open('outputs/directions_dict_5.31.2022.jsonl','a') as f:
    for d in directions:
        f.write(json.dumps(d)+'\n')

In [581]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [582]:
# Create Map
map = folium.Map(location=[conflicts.lat.mean(),conflicts.lon.mean()], zoom_start=6)

# Plot conflict starting points
for kk, start in conflicts[2:].iterrows():
    start_m = folium.Marker([start.lat, start.lon], popup=start['location'], 
                            icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
    start_m.add_to(map)
    
# Plot ending locations
for kk, vv in crossings.iterrows():
    # only plot the crossing if it's actually used
    if vv['#name'] in routes.crossing.unique():
        icon = 'glyphicon glyphicon-road'
        color = 'orange'
        popup_text = f"<b>Crossing Name: </b>{vv['#name']}<br>"\
                     f"<b>Country: </b>{vv['country']}"
        popup = folium.Popup(popup_text, max_width=300,min_width=150)
        xing = folium.Marker([vv.lat, vv.lon], popup=popup, 
                              icon=folium.Icon(icon=icon, color=color))
        xing.add_to(map)

In [583]:
fg = folium.FeatureGroup("Routes")
for d in directions:
    d = d['directions']
    distance = d[0]['legs'][0]['distance']['text']
    duration = d[0]['legs'][0]['duration']['text']
    end_location = d[0]['conflict']
    end_country = d[0]['crossing_country']
    tooltip = f"Travel between <b>{d[0]['conflict']}</b> and <b>{d[0]['crossing']}, {d[0]['crossing_country']}"\
              f"</b> by {d[0]['mode']} is <b>"\
              f"{distance}</b> and takes <b>{duration}</b>."
    polyline_ = polyline.decode(d[0]['overview_polyline']['points'])
    if d[0]['mode'] == 'transit':
        color = '#7570b3'
    else:
        color = '#4A89F3'
    polyline_m = folium.PolyLine(polyline_, color=color, tooltip=tooltip, weight=3, highlight=True)
    polyline_m.add_to(fg)
        
fg.add_to(map)

# Add custom basemaps
basemaps['Google Satellite Hybrid'].add_to(map)
# basemaps['Esri Satellite'].add_to(map)
# basemaps['Google Satellite'].add_to(map)
basemaps['Google Maps'].add_to(map)

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(map)

In [584]:
def add_legend(map):
    legend_html = """
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@100;300;400&display=swap');
    </style>
    
     <div style="
     padding-left:5px; padding-top:5px;
     position: fixed; 
     bottom: 50px; left: 50px; width: 160px; height: 120px;   
     border:2px solid grey; z-index:9999; border-radius: 15px;
     
     background-color:white;
     opacity: .85;
     
     font-size:14px;
     font-weight: bold;
     font-family: 'Roboto', sans-serif;
     ">

     <div class="awesome-marker-icon-darkred awesome-marker" style="margin-top: 10px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-fire icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:20px">Conflict Area</div>
     
     <div class="awesome-marker-icon-orange awesome-marker" style="margin-top: 60px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-road icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:25px">Border Crossing</div>     
     
     
      </div> """.format( title = "Legend html")
    map.get_root().html.add_child(folium.Element( legend_html ))
    return map

In [585]:
map = add_legend(map)

In [586]:
display(map)

In [587]:
# save map
map.save('mapping/ego_map.html')